Installing Packages

In [5]:
!pip install import-ipynb

In [6]:
!pip install ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Mounting Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Changing to required directory

In [8]:
%cd "/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation"

/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation


Importing required libraries

In [9]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pickle
import pandas as pd
import os
import import_ipynb

In [10]:
%run modifiedModel.ipynb

importing Jupyter notebook from modified_audio_to_text_dali.ipynb


AttributeError: ignored

In [11]:
import modifiedModel

importing Jupyter notebook from modifiedModel.ipynb
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


KeyError: ignored

In [19]:
# import ModifiedModel

ModuleNotFoundError: ignored

In [11]:
# import modifiedmodel

ModuleNotFoundError: ignored

In [10]:
# from modifiedModel.ipynb import ModifiedModel

ModuleNotFoundError: ignored

Defining custom Dataset

In [36]:
class TeacherDataset(Dataset):
  def __init__(self, logits, feature_maps):
    self.teacher_logits = logits
    self.teacher_feature_map = feature_maps

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    # indexing layer
    sample = {"teacher_logits": teacher_logits, "teacher_feature_map": teacher_feature_map}
    return sample

  def __len__(self):
    return len(self.teacher_logits)

Creating a List of Paths to Audio Files

In [37]:
# Paths to Train & Validation Sets
# os.chdir('/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation/slices/train')
# directory = 'recordings2'
# manifest_dir_url = '/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation/slices/train/recordings2'
# list_of_train_paths = []

# for filename in os.listdir(directory):
#     list_of_train_paths.append(f"{manifest_dir_url}/{filename}")

# os.chdir('/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation/slices/dev')
# directory = 'recordings2'
# manifest_dir_url = '/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation/slices/dev/recordings2'
# list_of_dev_paths = []

# for filename in os.listdir(directory):
#     list_of_dev_paths.append(f"{manifest_dir_url}/{filename}")

# list_of_all_paths = list_of_train_paths + list_of_dev_paths
# print(list_of_all_paths)

# Paths to Test Set
os.chdir('/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation/slices/test')
directory = 'recordings2'
manifest_dir_url = '/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation/slices/test/recordings2'
list_of_paths = []

for filename in os.listdir(directory):
    list_of_paths.append(f"{manifest_dir_url}/{filename}")

Initialising Modified Model

In [38]:
# Object Creation
teacher_model = ModifiedModel.from_pretrained("stt_en_conformer_ctc_large")

[NeMo I 2022-06-24 07:55:14 save_restore_connector:243] Model ModifiedModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.9.0/stt_en_conformer_ctc_large/010120d9959425c7862c9843960b3235/stt_en_conformer_ctc_large.nemo.


Getting Logits & Attention Maps

In [22]:
# Getting Teacher Model's Softmax Outputs
teacher_logits = teacher_model.transcribe(paths2audio_files=list_of_paths, batch_size = 4, logprobs = True)
# Getting Teacher Model's SAB Layer Outputs as Feature Maps
teacher_feature_map = teacher_model.transcribe(paths2audio_files=list_of_paths, batch_size = 4, return_self_attention_outputs = True)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2022-06-24 06:36:23 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Writing Data to Store it on Disk

In [39]:
# Writing objects to files to store on disk
# Writing teacher_logits
file = "teacher_logits.pkl"
file_obj = open(file, "wb") # write binary
pickle.dump(teacher_logits, file_obj)
file_obj.close()

# Writing teacher_feature_map
file = "teacher_feature_map.pkl"
file_obj = open(file, "wb") 
pickle.dump(teacher_feature_map, file_obj)
file_obj.close()

Loading Data from Disk

In [40]:
# Loading objects from files to restore their values to memory
# Loading teacher_logits
file = "teacher_logits.pkl"
file_obj = open(file, "rb") 
teacher_logits = pickle.load(file_obj)

# Loading teacher_feature_map
file = "teacher_feature_map.pkl"
file_obj = open(file, "rb") # read binary
teacher_feature_map = pickle.load(file_obj)

Initialising custom Dataset Object

In [41]:
# create Pandas DataFrame
teacher_df = pd.DataFrame({'teacher_logits': teacher_logits, 'teacher_feature_map': teacher_feature_map})
# define data set object
teacher_dataset = TeacherDataset(teacher_df['teacher_logits'], teacher_df['teacher_feature_map'])
# Display logits and maps of complete dataset.
print('\nFirst iteration of data set: ', next(iter(teacher_dataset)), '\n')
# Print how many items are in the data set
print('Length of data set: ', len(teacher_dataset), '\n')
# Print entire data set
print('Entire data set: ', list(DataLoader(teacher_dataset)), '\n') # Default Batch Size = 1

Streaming output truncated to the last 5000 lines.




               [[[[[[ 1.4097, -1.3339, -1.9399,  ...,  2.6370,  1.4448,
                     -1.4317],
                    [ 1.4730, -2.1186, -3.2685,  ...,  2.0479,  0.6359,
                     -2.0608],
                    [-0.0083, -2.9336, -0.9449,  ...,  2.5593, -0.7941,
                     -4.9364],
                    ...,
                    [ 0.7340, -2.7489, -1.3644,  ...,  1.9354, -0.1606,
                     -0.3265],
                    [ 0.9585, -2.6012, -2.0201,  ...,  3.0241, -0.2258,
                     -1.6080],
                    [ 1.9036, -2.8569, -1.7629,  ...,  2.7730,  0.1416,
                     -0.0548]],

                   [[ 1.4097, -1.3339, -1.9399,  ...,  2.6370,  1.4448,
                     -1.4317],
                    [ 1.4730, -2.1186, -3.2685,  ...,  2.0479,  0.6359,
                     -2.0608],
                    [-0.0083, -2.9336, -0.9449,  ...,  2.5593, -0.7941,
                     -

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Initialising DataLoader & iterating through the Data

In [42]:
teacher_dataloader = DataLoader(teacher_dataset, batch_size=2) 
# Below code will run 3 times because batch size = 4 and total samples = 9 (because: ceil(9/4) = 3)
# It would have run only once, had batch size been >= 9
for (idx, batch) in enumerate(teacher_dataloader):
  teacher_logits = batch["teacher_logits"]
  teacher_feature_map = batch["teacher_feature_map"]
  print("\nLOGITS:\n")
  print(teacher_logits)
  print("\nMAPS:\n")
  print(teacher_feature_map)

Streaming output truncated to the last 5000 lines.
                       2.3122],
                     [ 0.5561,  0.5739, -2.1535,  ...,  1.7159, -1.0840,
                       1.0452],
                     [ 1.1693,  0.6364, -1.5328,  ...,  1.9238, -0.4133,
                       1.5753]],

                    [[ 2.5782, -0.1753, -1.5312,  ...,  0.8718,  1.0030,
                       1.2090],
                     [ 2.2325,  0.5221, -0.3557,  ..., -0.2389,  0.5870,
                       2.6561],
                     [ 0.0064,  0.3808,  2.4641,  ...,  1.4150,  3.5415,
                       1.8283],
                     ...,
                     [ 0.1501, -0.8091, -2.5160,  ...,  0.9341, -0.9262,
                       2.3122],
                     [ 0.5561,  0.5739, -2.1535,  ...,  1.7159, -1.0840,
                       1.0452],
                     [ 1.1693,  0.6364, -1.5328,  ...,  1.9238, -0.4133,
                       1.5753]],

                    [[ 2.5782, -0.1753, -1.5312

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

